In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM 
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import pickle

In [3]:
raw_data = pd.read_csv('2015_labeled_and_time_normalize.csv').drop(['Unnamed: 0', 'Timestamp'], axis=1)

In [3]:
headers = raw_data.columns

In [ ]:
train_split_value = 250_000
val_split_values = 50_000

In [4]:
raw_data

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,2,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,2,1,251.1535,0.865024,189.0220,0.000000,1,1,1,Normal
449915,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal
449916,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal
449917,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,Normal


In [5]:
y = raw_data['Normal/Attack']

In [6]:
data = raw_data.drop(['Normal/Attack'], axis=1)

In [7]:
#нормализация
data_headers = data.columns
normalize_data =  pd.DataFrame(preprocessing.normalize(data), columns=data_headers)

In [70]:
normalize_data

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,0.001538,0.331411,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000195,0.001268,0.000634,0.159013,0.001046,0.120179,8.123024e-08,0.000634,0.000634,0.000634
1,0.001551,0.331436,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000195,0.001268,0.000634,0.159013,0.001046,0.120230,8.123025e-08,0.000634,0.000634,0.000634
2,0.001578,0.331392,0.001268,0.001268,0.000634,0.166072,0.005321,0.208295,0.001548,0.001268,...,0.000196,0.001268,0.000634,0.159014,0.001046,0.120223,8.122555e-08,0.000634,0.000634,0.000634
3,0.001606,0.331431,0.001268,0.001268,0.000634,0.166054,0.005320,0.208273,0.001548,0.001268,...,0.000196,0.001268,0.000634,0.158997,0.001046,0.120169,8.121678e-08,0.000634,0.000634,0.000634
4,0.001628,0.331635,0.001267,0.001267,0.000634,0.165994,0.005318,0.208198,0.001548,0.001267,...,0.000196,0.001267,0.000634,0.158940,0.001045,0.120055,8.118763e-08,0.000634,0.000634,0.000634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,0.001625,0.329704,0.001269,0.001269,0.000635,0.106674,0.005482,0.191599,0.001561,0.001269,...,0.000195,0.001269,0.000635,0.159381,0.000549,0.119953,0.000000e+00,0.000635,0.000635,0.000635
449915,0.001617,0.330153,0.001269,0.001269,0.000634,0.106642,0.005480,0.191542,0.001560,0.001269,...,0.000194,0.001269,0.000634,0.159283,0.000549,0.119856,0.000000e+00,0.000634,0.000634,0.000634
449916,0.001606,0.330337,0.001269,0.001269,0.000634,0.106645,0.005481,0.191547,0.001561,0.001269,...,0.000194,0.001269,0.000634,0.159287,0.000549,0.119859,0.000000e+00,0.000634,0.000634,0.000634
449917,0.001599,0.330315,0.001269,0.001269,0.000634,0.106630,0.005480,0.191520,0.001561,0.001269,...,0.000195,0.001269,0.000634,0.159265,0.000549,0.119842,0.000000e+00,0.000634,0.000634,0.000634


In [8]:
x_train = normalize_data[y == 0.0].values[:train_split_value]

In [13]:
val_data = normalize_data[y == 0][train_split_value:train_split_value+val_split_values]

In [73]:
# определяем какой индекс у крайней записи из валидационной выборки 
train_test_split_values = val_data.index[-1]

In [ ]:
val_data = val_data.values

In [15]:
x_test = normalize_data.values[train_test_split_values:]

In [16]:
# svm и iforest вовзращают -1 если сэмпл аномальный и 1 если неаномальный
# функция приводит ответ модели к виду из дата сета: 0 - нет аномалии, 1 - аномалия
def prepare_predict(predict):
    for i in range(len(predict)):
        if predict[i] == 1:
            predict[i] = 0
        else:
            predict[i] = 1
    return predict

In [19]:
%%time
svm.fit(x_train)

[LibSVM]CPU times: user 11min 17s, sys: 540 ms, total: 11min 18s
Wall time: 11min 18s


OneClassSVM(gamma=0.0008, nu=0.00458, verbose=True)

In [ ]:
svm_predict = prepare_predict(svm.predict(x_test))

In [ ]:
with open('svm.pickle', 'wb') as f:
    pickle.dump(svm, f)

In [80]:
print(classification_report(y_test, svm_predict))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     95299
         1.0       0.81      0.33      0.47      4704

    accuracy                           0.97    100003
   macro avg       0.89      0.67      0.73    100003
weighted avg       0.96      0.97      0.96    100003

